In [23]:
# Imports necesarios
import pandas as pd
import requests

try: 
    from BeautifulSoup import BeautifulSoup
except ImportError:
    from bs4 import BeautifulSoup
    
# Funcion para obtener el HTML de un link
def get_html(url):
    r = requests.get(url)
    html = BeautifulSoup(r.text)
    return html.body

# Variables necesarias
filename = "jordan_review_for_e_rav.xlsx"
filas_a_procesar = 10000 #Cambiar a -1 para procesar todas

# Ids a buscar, vienen del HTML
search_product_url = { 'Anuncio Finalizado': 'w1-5-_msg'}

# Leo el archivo en el dataframe
df = pd.read_excel("../spreadsheets/" + filename)

if filas_a_procesar != -1:
    df = df.head(filas_a_procesar)

# Saco las columnas que no voy a usar
cols_to_drop = ['site_id', 'MPN (ignore)']
df = df.drop(cols_to_drop, axis="columns")

# Agrego el https:// a todos los links
df['product_line_url'] = df['product_line_url'].apply(lambda s: 'https://'+s)
df['product_url'] = df['product_url'].apply(lambda s: 'https://'+s)
df

,Product_line_id,Product_line_description,product_id,product_title (by seller),product_line_url,product_url,YES/NO
0,7117656342,"[Brand = Jordan, Model = Air Jordan 1, Silhoue...",264737435029,NIKE AIR JORDAN 1 RETRO + NEUTRAL GREY 2001 SI...,https://www.ebay.com/b/bn_7117656342,https://www.ebay.com/itm/264737435029,y
1,7117717160,"[Brand = Jordan, Model = Jordan 'Why Not?' ZER...",164677534195,NEW Nike Jordan Why Not? Zer0.4 FACETASM Unise...,https://www.ebay.com/b/bn_7117717160,https://www.ebay.com/itm/164677534195,y
2,7117656574,"[Brand = Jordan, Model = Air Jordan 13, Silhou...",293808623286,Nike Air Jordan 2010 Retro 13 Varsity White Re...,https://www.ebay.com/b/bn_7117656574,https://www.ebay.com/itm/293808623286,y
3,7117651224,"[Brand = Jordan, Model = Air Jordan 5, Silhoue...",114780501812,2021 Nike Air Jordan Retro 5 Low Chinese New Y...,https://www.ebay.com/b/bn_7117651224,https://www.ebay.com/itm/114780501812,y
4,7117643400,"[Brand = Jordan, Model = Air Jordan 10, Silhou...",334065851300,Size 10 - Jordan 10 Retro NYC 2016,https://www.ebay.com/b/bn_7117643400,https://www.ebay.com/itm/334065851300,y
5,7117647055,"[Brand = Jordan, Model = Air Jordan 2, Silhoue...",114883811619,1994-1995 Nike Air Jordan II 2 Low Retro Chica...,https://www.ebay.com/b/bn_7117647055,https://www.ebay.com/itm/114883811619,y
6,7117649456,"[Brand = Jordan, Model = Jordan Flight Remix, ...",184714584239,Air Jordan Flight Remix Wolf Grey/Infrared 23-...,https://www.ebay.com/b/bn_7117649456,https://www.ebay.com/itm/184714584239,y
7,7117657277,"[Brand = Jordan, Model = Air Jordan 1, Silhoue...",334002458984,Jordan 1 Low TD Shadow 2019 Men’s Size 10 av52...,https://www.ebay.com/b/bn_7117657277,https://www.ebay.com/itm/334002458984,y
8,7117648427,"[Brand = Jordan, Model = Air Jordan 1, Silhoue...",193887868653,Size 9 - Jordan 1 High Zoom Air Comfort Red,https://www.ebay.com/b/bn_7117648427,https://www.ebay.com/itm/193887868653,y
9,7117648509,"[Brand = Jordan, Model = Air Jordan 1, Silhoue...",333958246029,Size 8.5 - Jordan 1 Retro High OG Carmine DS S...,https://www.ebay.com/b/bn_7117648509,https://www.ebay.com/itm/333958246029,y


In [ ]:
# Voy a procesar todas las filas para buscar los IDs que quiero
listas = [] # Lista de 2 dimensiones para guardar Y
for _ in range(len(search_product_url)):
    listas.append([])

if filas_a_procesar != -1:
    df = df.head(filas_a_procesar)
    

# Voy a recorrer todas las filas 
for index, row in df.iterrows():
    print("Procesando fila %d"%(index+1), end='\r')
    # Obtengo la URL (product_url)
    html = get_html(row['product_url'])
    
    # Busco los ids en la url
    for index, value in enumerate(search_product_url.values()):
        match = html.find(id=value)
        if match is not None:     
            listas[index].append("SI")
        else:
            listas[index].append("")

#Agrego la/s columna nueva con los resultados
for index, (key, value) in enumerate(search_product_url.items()):
    df.loc[:,key] = listas[index]
    
# Guardo los resultado en una nueva planilla
result_filename = filename.replace('spreadsheets', 'results').split('.')[0] + '_processed.xlsx'
df.to_excel('../results/'+result_filename)
    
df

In [20]:
#Fin
print("Resultado guardados en results/%s"%(result_filename))

Resultado guardados en results/jordan_review_for_e_rav_processed.xlsx
